In [5]:
# We use uv for faster installation
!pip install uv
!uv pip install -q autogluon.timeseries --system
!uv pip uninstall -q torchaudio torchvision torchtext --system # fix incompatible package versions on Colab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import matplotlib.pyplot as plt

data = pd.read_csv('data/imputed_train.csv')
data['id'] = pd.to_datetime(data['id'])
data

c:\Users\User\anaconda3\envs\ag_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25
0,2020-01-01 00:00:00,42.900000,0.718000,15.7,73.1,64.4
1,2020-01-01 01:00:00,33.600000,0.587000,10.1,74.8,66.0
2,2020-01-01 02:00:00,29.300000,0.400655,5.1,51.0,44.9
3,2020-01-01 03:00:00,30.500000,0.246000,7.2,27.7,25.1
4,2020-01-01 04:00:00,29.300000,0.204000,8.3,15.3,13.6
...,...,...,...,...,...,...
40986,2024-09-03 18:00:00,17.713860,0.222000,55.1,12.0,5.3
40987,2024-09-03 19:00:00,21.932757,0.245000,48.2,13.4,7.0
40988,2024-09-03 20:00:00,23.265996,0.234000,44.5,12.4,7.1
40989,2024-09-03 21:00:00,33.122175,0.225000,25.9,10.6,5.4


In [ ]:
data_long = pd.melt(data, id_vars=['id'], value_vars=['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25'],
                    var_name='Pollutant', value_name='Value')
train_data = TimeSeriesDataFrame.from_data_frame(
    data_long,
    id_column="Pollutant",
    timestamp_column="id"
)
train_data.head()

Value
item_id    timestamp                 
valeur_NO2 2020-01-01 00:00:00   42.9
           2020-01-01 01:00:00   33.6
           2020-01-01 02:00:00   29.3
           2020-01-01 03:00:00   30.5
           2020-01-01 04:00:00   29.3

In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length=504,
    path="autogluon-m4-hourly",
    target="Value",
    eval_metric="MAE",
)

predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=600,
)

NameError: name 'train_data' is not defined

In [ ]:
prediction = predictor.predict(train_data)[["mean"]]
prediction

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


mean
item_id     timestamp                     
valeur_NO2  2024-09-03 23:00:00  20.151454
            2024-09-04 00:00:00  19.795188
            2024-09-04 01:00:00  18.223992
            2024-09-04 02:00:00  17.992103
            2024-09-04 03:00:00  19.417732
...                                    ...
valeur_PM25 2024-09-24 18:00:00   8.706389
            2024-09-24 19:00:00   8.764595
            2024-09-24 20:00:00   9.172983
            2024-09-24 21:00:00   8.609866
            2024-09-24 22:00:00   7.961546

[2520 rows x 1 columns]

In [ ]:
prediction_wide = prediction.reset_index().pivot(index='timestamp', columns='item_id', values='mean')
prediction_wide["id"] = pd.to_datetime(prediction_wide.index)

ImportError: cannot import name 'clean_for_submission' from 'utils.preprocessing' (c:\Users\User\Mon Drive\DSB\2A HEC\Time series\time_series_project\utils\preprocessing.py)

In [ ]:
from utils.preprocessing import clean_for_submission

submission = pd.read_csv('submissions/matt_autogluon_raw.csv')
clean_for_submission(submission).to_csv('submissions/matt_autogluon_raw.csv', index=False)

# With all features on Colab

In [6]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

data_train = pd.read_csv("/content/drive/MyDrive/DSB/2A HEC/Time series/time_series_project/data/train_preprocessed.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
data_train

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,is_holiday,is_jour_ferie,precipitation,wind_speed,...,Year,is_weekend,DayOfYear,HourOfDay,DayOfYear_sin,DayOfYear_cos,HourOfDay_sin,HourOfDay_cos,Weekday_sin,Weekday_cos
0,2020-01-01 00:00:00,42.9,0.718,15.7,73.1,64.4,1,0,0.0,1.5,...,2020,0,1,0,0.017213,0.999852,0.000000,1.000000e+00,0.974928,-0.222521
1,2020-01-01 01:00:00,33.6,0.587,10.1,74.8,66.0,1,0,0.0,2.6,...,2020,0,1,1,0.017213,0.999852,0.258819,9.659258e-01,0.974928,-0.222521
2,2020-01-01 02:00:00,29.3,NaN,5.1,51.0,44.9,1,0,0.0,1.9,...,2020,0,1,2,0.017213,0.999852,0.500000,8.660254e-01,0.974928,-0.222521
3,2020-01-01 03:00:00,30.5,0.246,7.2,27.7,25.1,1,0,0.0,1.8,...,2020,0,1,3,0.017213,0.999852,0.707107,7.071068e-01,0.974928,-0.222521
4,2020-01-01 04:00:00,29.3,0.204,8.3,15.3,13.6,1,0,0.0,2.2,...,2020,0,1,4,0.017213,0.999852,0.866025,5.000000e-01,0.974928,-0.222521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40986,2024-09-03 18:00:00,NaN,0.222,55.1,12.0,5.3,0,0,0.0,1.2,...,2024,0,247,18,-0.895839,-0.444378,-1.000000,-1.836970e-16,0.781831,0.623490
40987,2024-09-03 19:00:00,NaN,0.245,48.2,13.4,7.0,0,0,0.0,1.7,...,2024,0,247,19,-0.895839,-0.444378,-0.965926,2.588190e-01,0.781831,0.623490
40988,2024-09-03 20:00:00,NaN,0.234,44.5,12.4,7.1,0,0,0.0,2.0,...,2024,0,247,20,-0.895839,-0.444378,-0.866025,5.000000e-01,0.781831,0.623490
40989,2024-09-03 21:00:00,NaN,0.225,25.9,10.6,5.4,0,0,0.0,1.4,...,2024,0,247,21,-0.895839,-0.444378,-0.707107,7.071068e-01,0.781831,0.623490


In [8]:
data_train["id"] = pd.to_datetime(data_train["id"])
known_covariates = ['is_holiday', 'precipitation',
       'wind_speed', 'temperature', 'humidity', 'pressure', 'visibility',
       'global_solar_radiation', 'Year', 'is_weekend', 'DayOfYear',
       'HourOfDay', 'DayOfYear_sin', 'DayOfYear_cos', 'HourOfDay_sin',
       'HourOfDay_cos', 'Weekday_sin', 'Weekday_cos']
targets = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']
data = data_train[['id'] + targets + known_covariates]
data

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,is_holiday,precipitation,wind_speed,temperature,...,Year,is_weekend,DayOfYear,HourOfDay,DayOfYear_sin,DayOfYear_cos,HourOfDay_sin,HourOfDay_cos,Weekday_sin,Weekday_cos
0,2020-01-01 00:00:00,42.9,0.718,15.7,73.1,64.4,1,0.0,1.5,0.8,...,2020,0,1,0,0.017213,0.999852,0.000000,1.000000e+00,0.974928,-0.222521
1,2020-01-01 01:00:00,33.6,0.587,10.1,74.8,66.0,1,0.0,2.6,-0.7,...,2020,0,1,1,0.017213,0.999852,0.258819,9.659258e-01,0.974928,-0.222521
2,2020-01-01 02:00:00,29.3,NaN,5.1,51.0,44.9,1,0.0,1.9,-0.3,...,2020,0,1,2,0.017213,0.999852,0.500000,8.660254e-01,0.974928,-0.222521
3,2020-01-01 03:00:00,30.5,0.246,7.2,27.7,25.1,1,0.0,1.8,0.0,...,2020,0,1,3,0.017213,0.999852,0.707107,7.071068e-01,0.974928,-0.222521
4,2020-01-01 04:00:00,29.3,0.204,8.3,15.3,13.6,1,0.0,2.2,0.3,...,2020,0,1,4,0.017213,0.999852,0.866025,5.000000e-01,0.974928,-0.222521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40986,2024-09-03 18:00:00,NaN,0.222,55.1,12.0,5.3,0,0.0,1.2,20.5,...,2024,0,247,18,-0.895839,-0.444378,-1.000000,-1.836970e-16,0.781831,0.623490
40987,2024-09-03 19:00:00,NaN,0.245,48.2,13.4,7.0,0,0.0,1.7,19.6,...,2024,0,247,19,-0.895839,-0.444378,-0.965926,2.588190e-01,0.781831,0.623490
40988,2024-09-03 20:00:00,NaN,0.234,44.5,12.4,7.1,0,0.0,2.0,18.4,...,2024,0,247,20,-0.895839,-0.444378,-0.866025,5.000000e-01,0.781831,0.623490
40989,2024-09-03 21:00:00,NaN,0.225,25.9,10.6,5.4,0,0.0,1.4,18.2,...,2024,0,247,21,-0.895839,-0.444378,-0.707107,7.071068e-01,0.781831,0.623490


In [9]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

train_data = pd.melt(data, id_vars=['id'] + known_covariates,
              value_vars=['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25'],
              var_name='Pollutant', value_name='target')
train_data = TimeSeriesDataFrame.from_data_frame(
    train_data,
    id_column="Pollutant",
    timestamp_column="id",
)
train_data.head()

is_holiday  precipitation  wind_speed  \
item_id    timestamp                                                    
valeur_NO2 2020-01-01 00:00:00           1            0.0         1.5   
           2020-01-01 01:00:00           1            0.0         2.6   
           2020-01-01 02:00:00           1            0.0         1.9   
           2020-01-01 03:00:00           1            0.0         1.8   
           2020-01-01 04:00:00           1            0.0         2.2   

                                temperature  humidity  pressure  visibility  \
item_id    timestamp                                                          
valeur_NO2 2020-01-01 00:00:00          0.8      94.0    1022.5      2139.0   
           2020-01-01 01:00:00         -0.7      96.0    1022.2       509.0   
           2020-01-01 02:00:00         -0.3      98.0    1022.2       779.0   
           2020-01-01 03:00:00          0.0      98.0    1021.9       377.0   
           2020-01-01 04:00:00          0.3      98.0    1021.6      1317.0   

                                global_solar_radiation  Year  is_weekend  \
item_id    timestamp                                                       
valeur_NO2 2020-01-01 00:00:00                     0.0  2020           0   
           2020-01-01 01:00:00                     0.0  2020           0   
           2020-01-01 02:00:00                     0.0  2020           0   
           2020-01-01 03:00:00                     0.0  2020           0   
           2020-01-01 04:00:00                     0.0  2020           0   

                                DayOfYear  HourOfDay  DayOfYear_sin  \
item_id    timestamp                                                  
valeur_NO2 2020-01-01 00:00:00          1          0       0.017213   
           2020-01-01 01:00:00          1          1       0.017213   
           2020-01-01 02:00:00          1          2       0.017213   
           2020-01-01 03:00:00          1          3       0.017213   
           2020-01-01 04:00:00          1          4       0.017213   

                                DayOfYear_cos  HourOfDay_sin  HourOfDay_cos  \
item_id    timestamp                                                          
valeur_NO2 2020-01-01 00:00:00       0.999852       0.000000       1.000000   
           2020-01-01 01:00:00       0.999852       0.258819       0.965926   
           2020-01-01 02:00:00       0.999852       0.500000       0.866025   
           2020-01-01 03:00:00       0.999852       0.707107       0.707107   
           2020-01-01 04:00:00       0.999852       0.866025       0.500000   

                                Weekday_sin  Weekday_cos  target  
item_id    timestamp                                              
valeur_NO2 2020-01-01 00:00:00     0.974928    -0.222521    42.9  
           2020-01-01 01:00:00     0.974928    -0.222521    33.6  
           2020-01-01 02:00:00     0.974928    -0.222521    29.3  
           2020-01-01 03:00:00     0.974928    -0.222521    30.5  
           2020-01-01 04:00:00     0.974928    -0.222521    29.3

In [10]:
predictor = TimeSeriesPredictor(
    prediction_length=504,
    target="target",
    eval_metric="MAE",
    known_covariates_names=known_covariates
)

predictor.fit(
    train_data,
    presets="high_quality",
    time_limit=900,
)

Beginning AutoGluon training... Time limit = 900s
AutoGluon will save models to 'AutogluonModels/ag-20241030_110306'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
GPU Count:          1
Memory Avail:       11.04 GB / 12.67 GB (87.1%)
Disk Space Avail:   74.48 GB / 112.64 GB (66.1%)
Setting presets to: high_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAE,
 'hyperparameters': 'default',
 'known_covariates_names': ['is_holiday',
                            'precipitation',
                            'wind_speed',
                            'temperature',
                            'humidity',
                            'pressure',
                            'visibility',
                            'global_solar_radiation',
                           

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/806M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

	-5.7754       = Validation score (-MAE)
	0.28    s     = Training runtime
	92.60   s     = Validation (prediction) runtime
Training timeseries model TemporalFusionTransformer. Training for up to 76.7s of the 306.9s of remaining time.
	-7.7136       = Validation score (-MAE)
	76.01   s     = Training runtime
	0.68    s     = Validation (prediction) runtime
Training timeseries model DeepAR. Training for up to 76.7s of the 230.1s of remaining time.
	-11.0324      = Validation score (-MAE)
	72.90   s     = Training runtime
	3.14    s     = Validation (prediction) runtime
Training timeseries model PatchTST. Training for up to 77.0s of the 154.1s of remaining time.
	-5.7189       = Validation score (-MAE)
	70.06   s     = Training runtime
	0.18    s     = Validation (prediction) runtime
Fitting simple weighted ensemble.
	Ensemble weights: {'AutoARIMA': 0.29, 'Chronos[base]': 0.01, 'DirectTabular': 0.55, 'DynamicOptimizedTheta': 0.08, 'NPTS': 0.05, 'SeasonalNaive': 0.02}
	-4.9064       = Val

In [27]:
test_data = pd.read_csv("/content/drive/MyDrive/DSB/2A HEC/Time series/time_series_project/data/test_preprocessed.csv")
test_data = pd.concat([test_data.assign(Pollutant=i) for i in targets], ignore_index=True)
test_data = TimeSeriesDataFrame.from_data_frame(
    test_data,
    id_column="Pollutant",
    timestamp_column="id",
)
predictor.predict(train_data, known_covariates=test_data)

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


mean        0.1        0.2        0.3  \
item_id     timestamp                                                         
valeur_NO2  2024-09-03 23:00:00  22.611733  13.437139  16.531632  18.827578   
            2024-09-04 00:00:00  19.800182   7.036595  11.407133  14.538802   
            2024-09-04 01:00:00  19.370732   4.046525   9.292860  13.041102   
            2024-09-04 02:00:00  18.436316   1.243021   7.052654  11.354644   
            2024-09-04 03:00:00  17.902708  -1.191955   5.282306  10.048133   
...                                    ...        ...        ...        ...   
valeur_PM25 2024-09-24 18:00:00   7.826760 -45.980547 -27.568358 -14.174674   
            2024-09-24 19:00:00   9.189113 -44.790441 -26.329397 -12.901951   
            2024-09-24 20:00:00   8.437398 -45.703173 -27.239343 -13.747260   
            2024-09-24 21:00:00   8.188633 -45.675488 -27.229332 -13.994358   
            2024-09-24 22:00:00   7.694221 -45.915577 -27.503748 -14.228452   

                                       0.4        0.5        0.6        0.7  \
item_id     timestamp                                                         
valeur_NO2  2024-09-03 23:00:00  20.797377  22.611733  24.464543  26.464547   
            2024-09-04 00:00:00  17.265796  19.800182  22.326897  25.059168   
            2024-09-04 01:00:00  16.301945  19.370732  22.452286  25.705182   
            2024-09-04 02:00:00  15.009139  18.436316  21.922969  25.652009   
            2024-09-04 03:00:00  14.094485  17.902708  21.733199  25.841287   
...                                    ...        ...        ...        ...   
valeur_PM25 2024-09-24 18:00:00  -2.957134   7.826760  18.406929  29.759384   
            2024-09-24 19:00:00  -1.486491   9.189113  19.816406  31.247489   
            2024-09-24 20:00:00  -2.269757   8.437398  19.046210  30.396158   
            2024-09-24 21:00:00  -2.453930   8.188633  18.839796  30.057274   
            2024-09-24 22:00:00  -2.886212   7.694221  18.407828  29.532672   

                                       0.8        0.9  
item_id     timestamp                                  
valeur_NO2  2024-09-03 23:00:00  28.739840  32.414111  
            2024-09-04 00:00:00  28.331751  33.024340  
            2024-09-04 01:00:00  29.899168  35.729475  
            2024-09-04 02:00:00  30.121890  36.187972  
            2024-09-04 03:00:00  30.689619  37.711460  
...                                    ...        ...  
valeur_PM25 2024-09-24 18:00:00  43.242327  62.054992  
            2024-09-24 19:00:00  44.942656  63.633676  
            2024-09-24 20:00:00  44.105255  62.923873  
            2024-09-24 21:00:00  43.610227  62.265259  
            2024-09-24 22:00:00  43.012745  61.475812  

[2520 rows x 10 columns]

In [ ]:
predictor.predict(train_data, known_covariates=test_data[known_covariates])

In [16]:
train_data.info()

<class 'autogluon.timeseries.dataset.ts_dataframe.TimeSeriesDataFrame'>
MultiIndex: 204955 entries, ('valeur_NO2', Timestamp('2020-01-01 00:00:00')) to ('valeur_PM25', Timestamp('2024-09-03 22:00:00'))
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   is_holiday              204955 non-null  int64  
 1   precipitation           204955 non-null  float64
 2   wind_speed              204955 non-null  float64
 3   temperature             204955 non-null  float64
 4   humidity                204955 non-null  float64
 5   pressure                204955 non-null  float64
 6   visibility              204955 non-null  float64
 7   global_solar_radiation  204955 non-null  float64
 8   Year                    204955 non-null  int64  
 9   is_weekend              204955 non-null  int64  
 10  DayOfYear               204955 non-null  int64  
 11  HourOfDay               204955 non-null  int64  
 12  DayO